In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 18.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 17.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 13.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
!pip install tqdm

In [2]:
#from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
#from peft import LoraConfig
#from trl import SFTConfig, SFTTrainer
import pandas as pd
from tqdm import tqdm

In [37]:
test_df = pd.read_csv('./test.csv')

In [38]:
test_df.head()

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%)
0,1_20240825 00,1,20240825 00,26.5,0.0,0.7,80.0
1,1_20240825 01,1,20240825 01,26.1,0.0,0.0,80.0
2,1_20240825 02,1,20240825 02,25.9,0.0,0.3,83.0
3,1_20240825 03,1,20240825 03,25.7,0.0,1.1,83.0
4,1_20240825 04,1,20240825 04,25.5,0.0,1.0,86.0


In [39]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num_date_time  16800 non-null  object 
 1   건물번호           16800 non-null  int64  
 2   일시             16800 non-null  object 
 3   기온(°C)         16800 non-null  float64
 4   강수량(mm)        16800 non-null  float64
 5   풍속(m/s)        16800 non-null  float64
 6   습도(%)          16800 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 918.9+ KB


In [40]:
system_message = """
너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45
"""

question = """
건물번호: {건물번호}, 일자: {일자}, 시간: {시간}, 기온: {기온}, 강수량: {강수량}, 풍속: {풍속}, 습도: {습도}
"""

answer = """
전력소비량(kWh): 
"""

In [41]:
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message,
            },
            {
                "role": "user",
                "content": question.format(
                    건물번호=sample['건물번호'],
                    일자=int(sample['일시'][0:8]),  # '20250815 10' 중 '20250815' 추출 후 int 변환
                    시간=int(sample['일시'][9:11]), # '20250815 10' 중 '10' 추출 후 int 변환
                    기온=sample['기온(°C)'],
                    강수량=sample['강수량(mm)'],
                    풍속=sample['풍속(m/s)'],
                    습도=sample['습도(%)'],
                    ),
            },
            {
                "role": "assistant",
                "content": answer
            },
        ],
    }



In [42]:
print(format_data(test_df.iloc[0]))

{'messages': [{'role': 'system', 'content': '\n너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.\n그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.\n주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.\n오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.\n\n[예시]\nuser\n건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0\nassistant\n전력소비량(kWh): 123.45\n'}, {'role': 'user', 'content': '\n건물번호: 1, 일자: 20240825, 시간: 0, 기온: 26.5, 강수량: 0.0, 풍속: 0.7, 습도: 80.0\n'}, {'role': 'assistant', 'content': '\n전력소비량(kWh): \n'}]}


In [43]:
# 빈 리스트 생성
test_dataset = []

# test_data 모든 행을 순회하며 format_data 함수 적용
for index, row in tqdm(test_df.iterrows()):
  test_dataset.append(format_data(row))

16800it [00:00, 21177.45it/s]


In [44]:

print(type(test_dataset), 'len=', len(test_dataset))


<class 'list'> len= 16800


In [45]:
test_dataset[0]

{'messages': [{'role': 'system',
   'content': '\n너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.\n그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.\n주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.\n오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.\n\n[예시]\nuser\n건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0\nassistant\n전력소비량(kWh): 123.45\n'},
  {'role': 'user',
   'content': '\n건물번호: 1, 일자: 20240825, 시간: 0, 기온: 26.5, 강수량: 0.0, 풍속: 0.7, 습도: 80.0\n'},
  {'role': 'assistant', 'content': '\n전력소비량(kWh): \n'}]}

In [17]:
model_id = "mypsyche98/Qwen3-4B-Instruct-2507_LoRA_20250818_merged"

In [18]:

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

...

model-00002-of-00006.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/218 [00:00<?, ?B/s]

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [46]:
text = tokenizer.apply_chat_template(
    test_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45
<|im_end|>
<|im_start|>user

건물번호: 1, 일자: 20240825, 시간: 0, 기온: 26.5, 강수량: 0.0, 풍속: 0.7, 습도: 80.0
<|im_end|>
<|im_start|>assistant
<think>

</think>

전력소비량(kWh): 
<|im_end|>



In [47]:
prompt_lst = []

for prompt in tqdm(test_dataset):
    text = tokenizer.apply_chat_template(
        prompt["messages"], tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    prompt_lst.append(input)

100%|██████████| 16800/16800 [00:01<00:00, 8573.63it/s]


In [48]:
print(prompt_lst[42])

<|im_start|>system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45
<|im_end|>
<|im_start|>user

건물번호: 1, 일자: 20240826, 시간: 18, 기온: 29.3, 강수량: 0.0, 풍속: 1.6, 습도: 74.0
<|im_end|>
<|im_start|>assistant


In [49]:
import numpy as np

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    smape_val = np.mean(np.where(denominator == 0, 0, np.abs(y_true - y_pred) / denominator)) * 100
    return smape_val

In [50]:
# 모델을 평가 모드로 전환 (매우 중요)
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [51]:
# ----------------- 4. 모델 예측 및 값 추출 -----------------
true_values = []
predicted_values = []
count = -1
zero_list = []

before = 0

for prompt in tqdm(prompt_lst, total=len(prompt_lst), desc="Evaluating model"):
    #count += 1
    # 토큰화 및 모델 예측
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            #max_new_tokens=20,
            pad_token_id=tokenizer.eos_token_id
        )

    # 예측된 텍스트 디코딩
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 예측된 텍스트에서 '전력소비량(kWh): ' 이후의 숫자만 추출
    try:
        # 모델이 생성한 전체 텍스트에서 'assistant' 부분만 추출
        predicted_part = predicted_text.split('assistant\n')[-1]
        predicted_power_consumption = float(predicted_part.split('전력소비량(kWh): ')[-1].strip())
        before = predicted_power_consumption
    except (ValueError, IndexError) as e:  # <-- 예외 객체를 변수 'e'로 받습니다.
        # 오류 종류와 상세 메시지를 출력
        #print(f"{count} 오류 발생: {type(e).__name__}")
        #print(f"{count} 오류 메시지: {e}")
        #print("predicted_text=",predicted_text)
        # 숫자를 찾지 못할 경우 0으로 처리 => 이전값으로 해볼까?
        predicted_power_consumption = before
        zero_list.append(count)

    predicted_values.append(predicted_power_consumption)
    


Evaluating model: 100%|██████████| 16800/16800 [3:21:57<00:00,  1.39it/s]  


In [55]:
predicted_values

[5040.3,
 4365.6,
 2889.0,
 3526.3,
 3637.7,
 5211.1,
 4731.3,
 5684.4,
 5414.9,
 5924.4,
 5397.7,
 6287.6,
 7303.1,
 5529.7,
 7515.9,
 6685.1,
 6532.9,
 6177.6,
 6177.6,
 5311.9,
 6517.6,
 5215.0,
 5577.3,
 5528.1,
 4729.7,
 3796.5,
 2924.3,
 3988.3,
 4414.5,
 4827.9,
 4796.5,
 4952.7,
 5697.9,
 6194.8,
 6069.8,
 5852.7,
 6126.1,
 6126.1,
 6400.5,
 6489.6,
 6132.6,
 6231.9,
 6041.7,
 5342.7,
 5340.3,
 4885.9,
 4669.6,
 4776.6,
 4816.2,
 2851.5,
 1573.5,
 3727.5,
 3629.8,
 4973.0,
 4973.0,
 5201.9,
 4650.3,
 4989.3,
 6019.6,
 6591.9,
 6268.3,
 5826.3,
 6398.3,
 6066.5,
 6099.0,
 6225.6,
 6038.0,
 6458.3,
 4928.1,
 5338.5,
 4833.0,
 4833.0,
 4833.0,
 1837.6,
 1807.6,
 3626.4,
 4486.3,
 4486.3,
 5616.8,
 4773.3,
 5563.1,
 5971.8,
 5971.8,
 6022.5,
 6192.3,
 6555.6,
 6555.6,
 7365.6,
 7492.6,
 6578.2,
 6637.7,
 6637.7,
 5880.4,
 5880.4,
 5439.7,
 5439.7,
 5462.1,
 1629.0,
 2997.9,
 3903.6,
 3913.3,
 4502.7,
 4869.7,
 5080.7,
 5080.7,
 5762.1,
 6759.6,
 6982.1,
 6982.1,
 7639.2,
 6439.8,
 

In [54]:
for x in range(len(predicted_values), 20):
    print("predicted_values[",x,"]=",predicted_values[x])


In [56]:
print(len(predicted_values))

16800


In [58]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = predicted_values

In [59]:
submission.head()

,num_date_time,answer
0,1_20240825 00,5040.3
1,1_20240825 01,4365.6
2,1_20240825 02,2889.0
3,1_20240825 03,3526.3
4,1_20240825 04,3637.7


In [60]:
submission.to_csv('./baseline_submission_20250818_01.csv', index=False)